In [ ]:
import gym
import torch
import numpy as np
from torch import nn
import random
import torch.nn.functional as F
import collections
from torch.optim.lr_scheduler import StepLR
from banana_env import BananaEnv
from music_env import MusicEnv
import music_utils

"""
Implementation of Double DQN for gym environments with discrete action space.
"""

print(torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

"""
The Q-Network has as input a state s and outputs the state-action values q(s,a_1), ..., q(s,a_n) for all n actions.
"""
class QNetwork(nn.Module):
    def __init__(self, action_dim, state_dim, hidden_dim):
        super(QNetwork, self).__init__()

        self.fc_1 = nn.Linear(state_dim, hidden_dim)
        self.fc_2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc_3 = nn.Linear(hidden_dim, action_dim)

    def forward(self, inp):

        x1 = F.leaky_relu(self.fc_1(inp))
        x1 = F.leaky_relu(self.fc_2(x1))
        x1 = self.fc_3(x1)

        return x1


"""
If the observations are images we use CNNs.
"""
class QNetworkCNN(nn.Module):
    def __init__(self, action_dim):
        super(QNetworkCNN, self).__init__()

        self.conv_1 = nn.Conv2d(3, 32, kernel_size=8, stride=4)
        self.conv_2 = nn.Conv2d(32, 64, kernel_size=4, stride=3)
        self.conv_3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.fc_1 = nn.Linear(8960, 512)
        self.fc_2 = nn.Linear(512, action_dim)

    def forward(self, inp):
        inp = inp.view((1, 3, 210, 160))
        x1 = F.relu(self.conv_1(inp))
        x1 = F.relu(self.conv_2(x1))
        x1 = F.relu(self.conv_3(x1))
        x1 = torch.flatten(x1, 1)
        x1 = F.leaky_relu(self.fc_1(x1))
        x1 = self.fc_2(x1)

        return x1


"""
memory to save the state, action, reward sequence from the current episode. 
"""
class Memory:
    def __init__(self, len):
        self.rewards = collections.deque(maxlen=len)
        self.state = collections.deque(maxlen=len)
        self.action = collections.deque(maxlen=len)
        self.is_done = collections.deque(maxlen=len)

    def update(self, state, action, reward, done):
        # if the episode is finished we do not save to new state. Otherwise we have more states per episode than rewards
        # and actions whcih leads to a mismatch when we sample from memory.
        if not done:
            self.state.append(state)
        self.action.append(action)
        self.rewards.append(reward)
        self.is_done.append(done)

    def sample(self, batch_size):
        """
        sample "batch_size" many (state, action, reward, next state, is_done) datapoints.
        """
        n = len(self.is_done)
        #print(n,batch_size)
        batch_size = min(n-1,batch_size)
        idx = random.sample(range(0, n-1), batch_size)

        #print(self.state)
        return torch.Tensor(self.state)[idx].to(device), torch.LongTensor(self.action)[idx].to(device), \
               torch.Tensor(self.state)[1+np.array(idx)].to(device), torch.Tensor(self.rewards)[idx].to(device), \
               torch.Tensor(self.is_done)[idx].to(device)

    def reset(self):
        self.rewards.clear()
        self.state.clear()
        self.action.clear()
        self.is_done.clear()


def select_action(model, env, state, eps):
    #print(state)
    state = torch.Tensor(state).to(device)
    with torch.no_grad():
        values = model(state)

    # select a random action wih probability eps
    if random.random() <= eps:
        action = env.chose_random_action() #np.random.randint(0, env.action_space.n)
    else:
        action = np.argmax(values.cpu().numpy())

    return action


def train(batch_size, current, target, optim, memory, gamma):

    states, actions, next_states, rewards, is_done = memory.sample(batch_size)

    q_values = current(states)

    next_q_values = current(next_states)
    next_q_state_values = target(next_states)

    q_value = q_values.gather(1, actions.unsqueeze(1)).squeeze(1)
    next_q_value = next_q_state_values.gather(1, torch.max(next_q_values, 1)[1].unsqueeze(1)).squeeze(1)
    expected_q_value = rewards + gamma * next_q_value * (1 - is_done)

    loss = (q_value - expected_q_value.detach()).pow(2).mean()

    optim.zero_grad()
    loss.backward()
    optim.step()


def evaluate(Qmodel, env, repeats):
    """
    Runs a greedy policy with respect to the current Q-Network for "repeats" many episodes. Returns the average
    episode reward.
    """
    Qmodel.eval()
    perform = 0
    for _ in range(repeats):
        state = env.reset()[0]
        done = False
        while not done:
            #print(state)
            #vectorstate = state[0]
            #print(state)
            state = torch.Tensor(state).to(device)
            with torch.no_grad():
                values = Qmodel(state)
            action = np.argmax(values.cpu().numpy())
            #print(env.step(action))
            state, reward, done, _, _ = env.step(action)
            perform += reward
    Qmodel.train()
    return perform/repeats


def compose(Qmodel,env,fn="Qmodel.mid",N=200):
    import copy
    Qmodel.eval()
    iinds = [[]]
    done = False
    env0 = copy.deepcopy(env)
    for random_init in [False,True]:
        env = copy.deepcopy(env0)
        env.random_init = random_init
        env.is_composing = True
        env.nr_of_notes_to_compose = N
        state = env.reset()[0]
        iinds = [[]]
        done = False        
        while not done:
            state = torch.Tensor(state).to(device)
            with torch.no_grad():
                values = Qmodel(state)
            action = np.argmax(values.cpu().numpy())
            state,reward,done,truncated, info = env.step(action)
            print("compose ",action,reward,info)
            iinds[0].append(info["keyPressed"])          
        music_utils.writePitches(fn+"-"+str(random_init)+".mid",iinds,tempo=70,instrument=[0],add21=False,start_at= [0],durationsInQuarterNotes=False)
    
def update_parameters(current_model, target_model):
    target_model.load_state_dict(current_model.state_dict())


def main(gamma=0.99, lr=1e-3, min_episodes=20, eps=1, eps_decay=0.995, eps_min=0.01, update_step=10, batch_size=64, update_repeats=50,
         num_episodes=3000, seed=42, max_memory_size=50000, lr_gamma=0.9, lr_step=100, measure_step=100,
         measure_repeats=100, hidden_dim=64, env=gym.make('CartPole-v1'), cnn=False, horizon=np.inf, render=True, render_step=50):
    """
    :param gamma: reward discount factor
    :param lr: learning rate for the Q-Network
    :param min_episodes: we wait "min_episodes" many episodes in order to aggregate enough data before starting to train
    :param eps: probability to take a random action during training
    :param eps_decay: after every episode "eps" is multiplied by "eps_decay" to reduces exploration over time
    :param eps_min: minimal value of "eps"
    :param update_step: after "update_step" many episodes the Q-Network is trained "update_repeats" many times with a
    batch of size "batch_size" from the memory.
    :param batch_size: see above
    :param update_repeats: see above
    :param num_episodes: the number of episodes played in total
    :param seed: random seed for reproducibility
    :param max_memory_size: size of the replay memory
    :param lr_gamma: learning rate decay for the Q-Network
    :param lr_step: every "lr_step" episodes we decay the learning rate
    :param measure_step: every "measure_step" episode the performance is measured
    :param measure_repeats: the amount of episodes played in to asses performance
    :param hidden_dim: hidden dimensions for the Q_network
    :param env_name: name of the gym environment
    :param cnn: set to "True" when using environments with image observations like "Pong-v0"
    :param horizon: number of steps taken in the environment before terminating the episode (prevents very long episodes)
    :param render: if "True" renders the environment every "render_step" episodes
    :param render_step: see above
    :return: the trained Q-Network and the measured performances
    """
    #env = gym.make(env_name)
    torch.manual_seed(seed)
    env.action_space.seed(seed)

    if cnn:
        Q_1 = QNetworkCNN(action_dim=env.action_space.n).to(device)
        Q_2 = QNetworkCNN(action_dim=env.action_space.n).to(device)
    else:
        print(env.observation_space.shape)
        Q_1 = QNetwork(action_dim=env.action_space.n, state_dim=env.observation_space.shape[0],
                                        hidden_dim=hidden_dim).to(device)
        Q_2 = QNetwork(action_dim=env.action_space.n, state_dim=env.observation_space.shape[0],
                                        hidden_dim=hidden_dim).to(device)
    # transfer parameters from Q_1 to Q_2
    update_parameters(Q_1, Q_2)

    # we only train Q_1
    for param in Q_2.parameters():
        param.requires_grad = False

    optimizer = torch.optim.Adam(Q_1.parameters(), lr=lr)
    scheduler = StepLR(optimizer, step_size=lr_step, gamma=lr_gamma)

    memory = Memory(max_memory_size)
    performance = []

    maxRewards = -np.inf
    
    for episode in range(num_episodes):
        # display the performance
        if episode % measure_step == 0:
            performance.append([episode, evaluate(Q_1, env, measure_repeats)])
            print("Episode: ", episode)
            print("rewards: ", performance[-1][1])
            print("lr: ", scheduler.get_last_lr()[0])
            print("eps: ", eps)
            if performance[-1][1] > maxRewards:
                maxRewards = performance[-1][1]
                import copy
                compose(Q_1,env,fn="Qmodel"+str(episode)+"-"+str(np.round(performance[-1][1],3))+".mid",N=200)

        state = env.reset()
        if type(state) == type((1,2)):
            state = state[0]
        memory.state.append(state)

        done = False
        i = 0
        while not done:
            i += 1
            #if type(state) == type((1,2)):
            #    state = state[0]
            action = select_action(Q_2, env, state, eps)
            state, reward, done, truncated,info = env.step(action)
            if i%100==0:
                print(reward,info)

            if i > horizon:
                done = True

            # render the environment if render == True
            if render and episode % render_step == 0:
                env.render()

            # save state, action, reward sequence
            memory.update(state, action, reward, done)

        if episode >= min_episodes and episode % update_step == 0:
            for _ in range(update_repeats):
                train(batch_size, Q_1, Q_2, optimizer, memory, gamma)

            # transfer new parameter from Q_1 to Q_2
            update_parameters(Q_1, Q_2)

        # update learning rate and eps
        #optimizer.step()
        scheduler.step()
        eps = max(eps*eps_decay, eps_min)

    return Q_1, performance


#if __name__ == '__main__':
env = MusicEnv(file="for_elise_by_beethoven.musicxml",maxNotes = 5,max_wrong_notes = 2,help_factor = 0.5)
#env = MusicEnv(file="reinforcement_learning.musicxml",maxNotes = 10)
horizon = np.inf
#horizon = 100
gamma = 0.99
#gamma = 0.05


Q_1,performance = main(gamma=gamma, lr=1e-3, min_episodes=20, eps=1, eps_decay=0.995, eps_min=0.01, update_step=10, batch_size=64, update_repeats=50,
         num_episodes=30000, seed=42, max_memory_size=50000, lr_gamma=0.9, lr_step=100, measure_step=10,
         measure_repeats=100, hidden_dim=640, env=env, cnn=False, horizon=horizon, render=True, render_step=50)
compose(Q_1,env,N=200)

[0.00390625, 0.0078125, 0.015625, 0.03125, 0.0625, 0.125, 0.25, 0.5, 1, 2, 4, 8, 0.005859375, 0.01171875, 0.0234375, 0.046875, 0.09375, 0.1875, 0.375, 0.75, 1.5, 3, 6, 12]


/home/orges/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


False
dimObs =  238
#Act =  97
computing vectors..
print computing knn..
..done
(238, 1)
Episode:  0
rewards:  -2.756809750418039
lr:  0.001
eps:  1
compose  7 -1.3784048752090226 {'keyPressed': (68, 0.0625, 76, False)}
compose  7 -1.3784048752090226 {'keyPressed': (68, 0.0625, 76, False)}
compose  7 -1.3784048752090226 {'keyPressed': (68, 0.0625, 76, False)}
compose  7 -1.3784048752090226 {'keyPressed': (68, 0.0625, 76, False)}
compose  7 -1.3784048752090226 {'keyPressed': (68, 0.0625, 76, False)}
compose  7 -1.3784048752090226 {'keyPressed': (68, 0.0625, 76, False)}
compose  7 -1.3784048752090226 {'keyPressed': (68, 0.0625, 76, False)}
compose  7 -1.3784048752090226 {'keyPressed': (68, 0.0625, 76, False)}
compose  7 -1.3784048752090226 {'keyPressed': (68, 0.0625, 76, False)}
compose  7 -1.3784048752090226 {'keyPressed': (68, 0.0625, 76, False)}
compose  7 -1.3784048752090226 {'keyPressed': (68, 0.0625, 76, False)}
compose  7 -1.3784048752090226 {'keyPressed': (68, 0.0625, 76, False)}

compose  7 -1.396424004376894 {'keyPressed': (68, 0.0625, 76, False)}
compose  7 -1.396424004376894 {'keyPressed': (68, 0.0625, 76, False)}
compose  7 -1.396424004376894 {'keyPressed': (68, 0.0625, 76, False)}
compose  7 -1.396424004376894 {'keyPressed': (68, 0.0625, 76, False)}
compose  7 -1.396424004376894 {'keyPressed': (68, 0.0625, 76, False)}
compose  7 -1.396424004376894 {'keyPressed': (68, 0.0625, 76, False)}
compose  7 -1.396424004376894 {'keyPressed': (68, 0.0625, 76, False)}
compose  7 -1.396424004376894 {'keyPressed': (68, 0.0625, 76, False)}
compose  7 -1.396424004376894 {'keyPressed': (68, 0.0625, 76, False)}
compose  7 -1.396424004376894 {'keyPressed': (68, 0.0625, 76, False)}
compose  7 -1.396424004376894 {'keyPressed': (68, 0.0625, 76, False)}
compose  7 -1.396424004376894 {'keyPressed': (68, 0.0625, 76, False)}
compose  7 -1.396424004376894 {'keyPressed': (68, 0.0625, 76, False)}
compose  7 -1.396424004376894 {'keyPressed': (68, 0.0625, 76, False)}
compose  7 -1.396424

/home/orges/.local/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


...

Episode:  770
rewards:  14.433075700070047
lr:  0.0004782969
eps:  0.021075222784267326
Episode:  780
rewards:  15.746098160783376
lr:  0.0004782969
eps:  0.020044857891939702
compose  3 -1.4111088087654469 {'keyPressed': (69, 0.0625, 70, False)}
compose  74 1.791759469228055 {'keyPressed': (64, 0.0625, 76, False)}
compose  11 1.3862943611198906 {'keyPressed': (69, 0.0625, 76, False)}
compose  49 -0.4140393356054125 {'keyPressed': (40, 0.0625, 70, False)}
compose  67 1.3862943611198906 {'keyPressed': (40, 0.0625, 64, False)}
compose  19 1.0986122886681098 {'keyPressed': (52, 0.0625, 64, False)}
compose  15 1.6094379124341003 {'keyPressed': (56, 0.0625, 70, False)}
compose  74 1.791759469228055 {'keyPressed': (64, 0.0625, 76, False)}
compose  7 1.0986122886681098 {'keyPressed': (68, 0.0625, 76, False)}
compose  21 1.6094379124341003 {'keyPressed': (71, 0.0625, 76, False)}
compose  78 1.9459101490553132 {'keyPressed': (45, 0.0625, 64, False)}
compose  78 -1.3557637102737479 {'keyPressed'

compose  94 -1.2255643729027517 {'keyPressed': (64, 0.0625, 83, False)}
compose  74 -1.1547005383792517 {'keyPressed': (64, 0.0625, 76, False)}
compose  3 -1.2875926129184283 {'keyPressed': (69, 0.0625, 70, False)}
compose  11 -1.224744871391589 {'keyPressed': (69, 0.0625, 76, False)}
compose  67 -1.4659886790209349 {'keyPressed': (40, 0.0625, 64, False)}
compose  19 -1.4079972089286537 {'keyPressed': (52, 0.0625, 64, False)}
compose  24 -1.4083086782851744 {'keyPressed': (56, 0.0625, 76, False)}
compose  15 -1.4632935693754139 {'keyPressed': (56, 0.0625, 70, False)}
compose  74 -1.1547005383792517 {'keyPressed': (64, 0.0625, 76, False)}
compose  7 -1.402378931197509 {'keyPressed': (68, 0.0625, 76, False)}
compose  21 -1.3743685418725535 {'keyPressed': (71, 0.0625, 76, False)}
compose  78 -0.5619514869490165 {'keyPressed': (45, 0.0625, 64, False)}
compose  78 -0.5619514869490165 {'keyPressed': (45, 0.0625, 64, False)}
compose  5 -1.3507632670119414 {'keyPressed': (52, 0.0625, 70, False

Episode:  790
rewards:  17.147119795682666
lr:  0.0004782969
eps:  0.019064867404770626
compose  74 1.791759469228055 {'keyPressed': (64, 0.0625, 76, False)}
compose  30 -0.41070025419420075 {'keyPressed': (69, 0.0625, 83, False)}
compose  11 1.3862943611198906 {'keyPressed': (69, 0.0625, 76, False)}
compose  49 -0.4140393356054125 {'keyPressed': (40, 0.0625, 70, False)}
compose  67 1.3862943611198906 {'keyPressed': (40, 0.0625, 64, False)}
compose  19 1.3862943611198906 {'keyPressed': (52, 0.0625, 64, False)}
compose  15 1.6094379124341003 {'keyPressed': (56, 0.0625, 70, False)}
compose  74 1.791759469228055 {'keyPressed': (64, 0.0625, 76, False)}
compose  7 1.0986122886681098 {'keyPressed': (68, 0.0625, 76, False)}
compose  21 1.6094379124341003 {'keyPressed': (71, 0.0625, 76, False)}
compose  78 1.9459101490553132 {'keyPressed': (45, 0.0625, 64, False)}
compose  78 -1.3557637102737479 {'keyPressed': (45, 0.0625, 64, False)}
compose  5 1.6094379124341003 {'keyPressed': (52, 0.0625, 7

compose  78 -1.1470786693528088 {'keyPressed': (45, 0.0625, 64, False)}
compose  5 -1.4111088087654469 {'keyPressed': (52, 0.0625, 70, False)}
compose  96 1.3862943611198906 {'keyPressed': (57, 0.0625, 76, False)}
compose  89 -1.4345364187925325 {'keyPressed': (64, 0.0625, 70, False)}
compose  74 -1.378404875209022 {'keyPressed': (64, 0.0625, 76, False)}
compose  11 -1.3662601021279466 {'keyPressed': (69, 0.0625, 76, False)}
compose  74 -1.378404875209022 {'keyPressed': (64, 0.0625, 76, False)}
compose  67 -1.5135354220117248 {'keyPressed': (40, 0.0625, 64, False)}
compose  19 -1.5052539565416228 {'keyPressed': (52, 0.0625, 64, False)}
compose  14 -1.5491933384829661 {'keyPressed': (56, 0.0625, 57, False)}
compose  15 -1.4345364187925322 {'keyPressed': (56, 0.0625, 70, False)}
compose  74 -1.378404875209022 {'keyPressed': (64, 0.0625, 76, False)}
compose  74 -1.378404875209022 {'keyPressed': (64, 0.0625, 76, False)}
compose  7 -1.3964240043768936 {'keyPressed': (68, 0.0625, 76, False)}

Episode:  800
rewards:  15.362118811544706
lr:  0.00043046721
eps:  0.018132788524664028
